In [ ]:
import os

from aavomics import database
from aavomics import aavomics
import anndata
import numpy
import pandas

import plotly.graph_objects as graph_objects
from plotly import offline as plotly

In [ ]:
droplet_classifier_adata = anndata.read_h5ad(os.path.join(database.DATA_PATH, "aavomics_mouse_cortex_2021_droplet_training_data.h5ad"))

In [ ]:
droplet_type_clusters = {droplet_type: [] for droplet_type in list(droplet_classifier_adata.obs["CCN202105041"].unique())}

for cluster in droplet_classifier_adata.obs["leiden_scVI"].unique():
    
    cluster_adata = droplet_classifier_adata[droplet_classifier_adata.obs["leiden_scVI"] == cluster].copy()

    droplet_types = list(cluster_adata.obs["CCN202105041"].unique())
    
    if len(droplet_types) > 1:
        
        droplet_types.remove("Multiplets")

    droplet_type = droplet_types[0]
    
    droplet_type_clusters[droplet_type].append(cluster)

In [ ]:
mt_genes = droplet_classifier_adata.var.loc[droplet_classifier_adata.var["Gene Name"].str.startswith("mt-")].index.values

In [ ]:
traces = []

for droplet_type, clusters in droplet_type_clusters.items():
    
    x = []
    y = []
    
    for cluster in clusters:
        
        cluster_adata = droplet_classifier_adata[droplet_classifier_adata.obs["leiden_scVI"] == cluster].copy()
        
        total_transcript_counts = numpy.array(cluster_adata.X.sum(axis=1)).flatten()
        
        mt_counts = numpy.array(cluster_adata[:, mt_genes].X.sum(axis=1)).flatten()
        
        mt_ratios = mt_counts/total_transcript_counts
        
        x.append(total_transcript_counts.mean())
        y.append(mt_ratios.mean())
    
    trace = graph_objects.Scatter(
        x=x,
        y=y,
        name=droplet_type,
        mode="markers"
    )
    
    traces.append(trace)

In [ ]:
layout = {}

layout["height"] = 500
layout["width"] = 700
layout["plot_bgcolor"] = "rgba(255, 255, 255, 0)"
layout["paper_bgcolor"] = "rgba(255, 255, 255, 0)"
layout["xaxis"] = {
    "title": "Number of Transcripts",
    "rangemode": "tozero",
    "gridcolor": "rgba(0, 0, 0, 0.25)",
    "zerolinecolor": "rgba(0, 0, 0, 0.25)",
}
layout["yaxis"] = {
    "title": "% Mitochondrial Transcripts",
    "rangemode": "tozero",
    "gridcolor": "rgba(0, 0, 0, 0.25)",
    "zerolinecolor": "rgba(0, 0, 0, 0.25)"
}

figure = graph_objects.Figure(data=traces, layout=layout)

plotly.iplot(figure)

In [ ]:
figure.write_image(os.path.join("out", "cell_typing_qc.svg"))